In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
# from datetime import datetime, timedelta

# import pandas as pd

# current_date = pd.to_datetime(datetime.utcnow())
# print(f"{current_date}")


# print(type(current_date))



In [4]:
from datetime import datetime
import pandas as pd
import pytz

# Get the current UTC time
current_date_utc = pd.to_datetime(datetime.utcnow())

# Convert it to EST
est_timezone = pytz.timezone('US/Eastern')
current_date_est = current_date_utc.tz_localize(pytz.utc).astimezone(est_timezone)

# Remove the timezone info
current_date_est_without_tz = current_date_est.tz_localize(None)

print(f"{current_date_est_without_tz}")


2025-03-04 22:49:34.704554


In [5]:
from datetime import timedelta
fetch_data_to = current_date_est_without_tz
fetch_data_from = current_date_est_without_tz - timedelta(days=29)


In [6]:
fetch_data_to

Timestamp('2025-03-04 22:49:34.704554')

In [7]:
fetch_data_from

Timestamp('2025-02-03 22:49:34.704554')

In [8]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    # Load and filter data for the historical period
    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from
    # Shift the data forward by 52 weeks to simulate recent data
    rides['pickup_datetime'] += timedelta(weeks=52)

    # Sort the data for consistency
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [9]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-02.
Loading data for 2024-02...
Total records: 3,007,526
Valid records: 2,954,709
Records dropped: 52,817 (1.76%)
Successfully processed data for 2024-02.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2024-03.
Loading data for 2024-03...
Total records: 3,582,628
Valid records: 3,518,066
Records dropped: 64,562 (1.80%)
Successfully processed data for 2024-03.
Combining all monthly data...
Data loading and processing complete!


In [10]:
rides

,pickup_datetime,pickup_location_id
813553,2025-02-12 16:25:44,2
1120254,2025-02-15 16:56:40,2
3969,2025-02-04 04:14:50,3
5570,2025-02-04 06:08:40,3
73323,2025-02-04 18:17:57,3
...,...,...
2943261,2025-03-04 22:40:06,263
2944906,2025-03-04 22:41:33,263
2945250,2025-03-04 22:43:25,263
2946944,2025-03-04 22:46:44,263


In [11]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [12]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-02-03 22:00:00,2,0
1,2025-02-03 23:00:00,2,0
2,2025-02-04 00:00:00,2,0
3,2025-02-04 01:00:00,2,0
4,2025-02-04 02:00:00,2,0
...,...,...,...
174942,2025-03-04 18:00:00,263,132
174943,2025-03-04 19:00:00,263,99
174944,2025-03-04 20:00:00,263,86
174945,2025-03-04 21:00:00,263,76


In [13]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174947 entries, 0 to 174946
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174947 non-null  datetime64[ns]
 1   pickup_location_id  174947 non-null  int16         
 2   rides               174947 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 2.0 MB


In [14]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)



2025-03-04 22:49:53,848 INFO: Initializing external client
2025-03-04 22:49:53,849 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 22:49:56,268 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215683


In [15]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 174947/174947 | Elapsed Time: 00:20 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1215683/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)